In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
import numpy as np
import os

# 03. 回归问题

##### 1. 数据准备
我们使用手写数据集`MNIST`，它包含了`0~9`共10种数字的手写图片，每种数字一共有7000张图片，采集自不同书写风格的真实手写图片，一共70000张图片。其中60000张图片作为训练集$\mathbb{D}^{\mathrm{train}}$用来训练模型，剩下10000张图片作为测试集$\mathbb{D}^{\mathrm{test}}$用来预测或者测试，训练集和测试集共同组成了整个`MNIST`数据集。

考虑到手写数字图片包含的信息比较简单，每张图片均被缩放到$28\times 28$的大小，同时只保留了灰度信息，因此，每张图片可以用$[h,w,1]$形状的张量来表示。

我们利用`TensorFlow`自动在线下载`MNIST`数据集，并转换为`Numpy`数组格式。

In [2]:
(x, y), (x_val, y_val) = datasets.mnist.load_data() # 加载 MNIST 数据集

x = 2*tf.convert_to_tensor(x, dtype=tf.float32)/255.-1 # 转换为浮点张量，并缩放到-1~1
y = tf.convert_to_tensor(y, dtype=tf.int32) # 转换为整形张量
y = tf.one_hot(y, depth=10) # one-hot 编码

(x.shape, y.shape)

(TensorShape([60000, 28, 28]), TensorShape([60000, 10]))

每一张图片的计算流程是通用的，我们在计算的过程中可以一次进行多张图片的计算，充分利用CPU或GPU的并行计算能力。我们用形状为$[h,w]$的矩阵来表示一张图片，对于多张图片来说，我们在前面添加一个数量维度(Dimension)，使用形状为$[b,h,w]$的张量来表示，其中$b$代表了批量(Batch Size)；多张彩色图片可以使用形状为$[b,h,w,c]$的张量来表示，其中$c$表示通道数量(Channel)，彩色图片$c=3$。通过`TensorFlow`的`Dataset`对象可以方便完成模型的批量训练，只需要调用`batch()`函数即可构建带`batch`功能的数据集对象。

In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)) # 构建数据集对象
train_dataset = train_dataset.batch(512) # 批量训练
train_dataset

<BatchDataset shapes: ((None, 28, 28), (None, 10)), types: (tf.float32, tf.float32)>

##### 2. 模型构建
我们使用2层的全连接模型，公式如下：
+ $h_1 = \mathrm{ReLU}(W_1x + b_1)$
+ $h_2 = \mathrm{ReLU}(W_2x + b_2)$
+ $o = W_3h_2 + b_3$

对于第一层模型来说，它接受的输入$x \in \mathcal{R}^{784}$，输出$h_1 \in \mathcal{R}^{256}$设计为长度为256的向量，我们不需要显式地编写$h_1 = \mathrm{ReLU}(W_1x + b_1)$的计算逻辑，在`TensorFlow`中通过一行代码即可实现：
```python
# 创建一层网络，设置输出节点数为256，激活函数类型为ReLU 
layers.Dense(256, activation='relu')
```

对于3层网络我们也可以快速搭建：

In [4]:
# 利用Sequential容器封装3个网络层，前网络层的输出默认作为下一层的输入
model = keras.Sequential([
    layers.Dense(256, activation='relu'), # 隐藏层 1
    layers.Dense(128, activation='relu'), # 隐藏层 2
    layers.Dense(10)]) # 输出层，输出节点数为 10

optimizer = optimizers.SGD(lr=0.01)

给定输入$x$，调用`model(x)`得到模型输出$o$后，通过`MSE`损失函数计算当前的误差$\mathcal{L}$：

In [5]:
with tf.GradientTape() as tape: # 构建梯度记录环境
    # 打平操作，[b, 28, 28] => [b, 784]
    x = tf.reshape(x, (-1, 28*28))
    # Step1. 得到模型输出 output [b, 784] => [b, 10]
    out = model(x)
    # [b] => [b, 10]
    y_onehot = y
    # 计算差的平方和，[b, 10]
    loss = tf.square(out-y_onehot)
    # 计算每个样本的平均误差，[b]
    loss = tf.reduce_sum(loss)/x.shape[0]
# Step3. 计算参数的梯度 w1, w2, w3, b1, b2, b3
grads = tape.gradient(loss, model.trainable_variables)
# w' = w - lr * grad，更新网络参数
optimizer.apply_gradients(zip(grads, model.trainable_variables))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

以上就是一个神经网络模型的构造过程，这里省略了测试部分，后面章节会涉及。